In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class FieldCropsPredictionModel(nn.Module):
		def __init__(self, input_dim, hidden_dim, layer_num=2):
				super(FieldCropsPredictionModel, self).__init__()
				self.filter_layer = nn.Linear(input_dim, 1, bias=True)
				self.hidden_layers = nn.ModuleList()
				self.output_layers = nn.Linear(hidden_dim, 1)

				for i in range(layer_num):
					self.hidden_layers.append(nn.Linear(12 if i == 0 else hidden_dim, hidden_dim))
				
		def forward(self, input):
			input = input.t()
			# 배치를 사용할 때
			# input = input.transpose(1, 2)
			out = self.filter_layer(input)
			for i, layer in enumerate(self.hidden_layers):
				out = layer(out)
			
			out = self.output_layers(out)

			return out

In [ ]:
def train(model, dataloader, criterion, optim, scheduler, num_epochs):
		model.train()
		train_loss_list = []
		for epoch in range(num_epochs):
				running_loss = 0.0
				
				for inputs, labels in dataloader:
						outputs = model(inputs)
						loss = criterion(outputs, labels)

						optim.zero_grad()
						loss.backward()
						optim.step()

						running_loss += loss
				
				scheduler.step()

				epoch_loss = running_loss / len(dataloader.dataset)
				train_loss_list.append(epoch_loss)
		
		return model, train_loss_list

In [ ]:
def validation(model, dataloader, criterion, num_epochs):
		model.eval()
		val_loss_list = []
		for epoch in range(num_epochs):
				running_loss = 0.0
				
				for inputs, labels in dataloader:
						outputs = model(inputs)
						loss = criterion(outputs, labels)

						running_loss += loss
				

				epoch_loss = running_loss / len(dataloader.dataset)
				val_loss_list.append(epoch_loss)
		
		return model, val_loss_list

In [ ]:
model = FieldCropsPredictionModel(10, 15, 2)
optimizer_ft = optim.Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = optim.lr_scheduler.StepLR(
    optimizer_ft, step_size=7, gamma=0.1)
criterion = nn.MSELoss()

In [ ]:
model_train, train_loss = train(
    model, train_loader, criterion, optimizer_ft, exp_lr_scheduler, 50)

model_val, val_loss = validation(
    model, val_loader, criterion, 50)